In [1]:
import os
from mocap_visualizer.generate_views import (    # organize this
    get_output_dir,
    prepare_videos
)
from mocap_visualizer.animated_phase_analysis import generate_all_animations
from mocap_visualizer.animated_merged_phase_analysis import animate_merged_phase_analysis
from mocap_visualizer.kinematic_visualizer import visualize_joint_position
from mocap_visualizer.video_layout import combine_views

In [2]:
# Configuration
filename = "BKO_E1_D5_01_Maraka"
bvh_file = filename + "_T"
start_time = 122.0
end_time = 125.0

# path to onsets and cycles csv files
cycles_csv_path = f"data/virtual_cycles/{filename}_C.csv"
onsets_csv_path = f"data/drum_onsets/{filename}.csv"

# prepare output directory
output_dir = get_output_dir(bvh_file, start_time, end_time)
output_fps = 24

### Generate Separate Drum distribution plots

In [ ]:
print("\nGenerating distribution plot videos...")

# generate animated distribution plots
generate_all_animations(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    save_dir=output_dir,
    figsize=(10, 3), dpi=200
)

### Generate merged distribution plot (DunDun, J1, J2)

In [ ]:
merged_save_path = "drum_combined.mp4"
animate_merged_phase_analysis(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    figsize=(10, 4), dpi=200,
    save_path=merged_save_path,
    save_dir=output_dir
)

### Generate Skeleton Videos + trimmed video_mix + audio

In [4]:

video_size = (640, 360)
# views_to_generate = ['front', 'right']

output_dir, view_videos = prepare_videos(
    filename=bvh_file,
    start_time=start_time,
    end_time=end_time,
    video_path= f"{filename}_pre_R_Mix.mp4",
    video_size=video_size,
    fps=output_fps
)


Generating front view...

BVH file information:
Frame rate: 240 Hz
Total frames: 39562
Total time: 164.84 seconds
Position columns: ['Time', 'Hips.X', 'Hips.Y', 'Hips.Z', 'LeftHip.X', 'LeftHip.Y', 'LeftHip.Z', 'LeftKnee.X', 'LeftKnee.Y', 'LeftKnee.Z', 'LeftAnkle.X', 'LeftAnkle.Y', 'LeftAnkle.Z', 'LeftToe.X', 'LeftToe.Y', 'LeftToe.Z', 'LeftToeEnd.X', 'LeftToeEnd.Y', 'LeftToeEnd.Z', 'RightHip.X', 'RightHip.Y', 'RightHip.Z', 'RightKnee.X', 'RightKnee.Y', 'RightKnee.Z', 'RightAnkle.X', 'RightAnkle.Y', 'RightAnkle.Z', 'RightToe.X', 'RightToe.Y', 'RightToe.Z', 'RightToeEnd.X', 'RightToeEnd.Y', 'RightToeEnd.Z', 'Chest.X', 'Chest.Y', 'Chest.Z', 'Chest2.X', 'Chest2.Y', 'Chest2.Z', 'Chest3.X', 'Chest3.Y', 'Chest3.Z', 'Chest4.X', 'Chest4.Y', 'Chest4.Z', 'LeftCollar.X', 'LeftCollar.Y', 'LeftCollar.Z', 'LeftShoulder.X', 'LeftShoulder.Y', 'LeftShoulder.Z', 'LeftElbow.X', 'LeftElbow.Y', 'LeftElbow.Z', 'LeftWrist.X', 'LeftWrist.Y', 'LeftWrist.Z', 'LeftWristEnd.X', 'LeftWristEnd.Y', 'LeftWristEnd.Z', 

### Generate animated kinematic plots

In [7]:
joint_name = "LeftAnkle"  
axis = 'y'

In [ ]:
# Generate joint position visualization

joint_video = os.path.join(output_dir, f"{joint_name}_{axis}_position.mp4")
if not os.path.exists(joint_video):
    visualize_joint_position(
        bvh_file=bvh_file + ".bvh",
        joint_name=joint_name,
        axis=axis,
        start_time=start_time,
        end_time=end_time,
        output_fps=output_fps,
        output_dir=output_dir,
        fig_size= (12, 4),  # Half height for joint visualization
        dpi= 200
    )

### Part 2: Combine the videos

In [5]:
def get_video_paths(output_dir, filename, joint_name="LeftAnkle", axis='y'):
    """Get dictionary mapping view names to their expected video paths"""
    return {
        'front': os.path.join(output_dir, "front_view.mp4"),
        'right': os.path.join(output_dir, "right_view.mp4"),
        # 'left': os.path.join(output_dir, "left_view.mp4"),
        # 'top': os.path.join(output_dir, "top_view.mp4"),
        
        'dundun': os.path.join(output_dir, "Dun.mp4"),
        'J1': os.path.join(output_dir, "J1.mp4"),
        'J2': os.path.join(output_dir, "J2.mp4"),
        'combined': os.path.join(output_dir, "drum_combined.mp4"),
        
        'joint_pos': os.path.join(output_dir, f"{joint_name}_{axis}_position.mp4"),
        'video_mix': os.path.join(output_dir, f"{filename}_pre_R_Mix_trimmed.mp4"),
        'audio': os.path.join(output_dir, f"{filename}_pre_R_Mix_trimmed_audio.mp3")
        
    }

In [ ]:
# Choose layout
layout = {
    'L1': [
        {'view': 'front', 'x': 0, 'y': 0, 'width': 320, 'height': 360},
        {'view': 'right', 'x': 320, 'y': 0, 'width': 320, 'height': 360},
        {'view': 'video_mix', 'x': 640, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'combined', 'x': 0, 'y': 360, 'width': 1280, 'height': 360}
    ],
    'L2': [
        {'view': 'front', 'x': 0, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'right', 'x': 640, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'joint_position', 'x': 0, 'y': 360, 'width': 1280, 'height': 360}
    ]
    }   

canvas_size = (1280, 720)   
joint_name = "LeftAnkle"  
axis = 'y'

layout_config = layout['L1']

# Get video paths
view_videos = get_video_paths(output_dir, filename, joint_name, axis)

# Check if we have all required videos for the layout
required_views = {item['view'] for item in layout_config}
missing_views = required_views - set(view_videos.keys())

if missing_views:
    print(f"Warning: Missing videos for views: {missing_views}")
    print("Please generate the videos first")


# Combine the videos
combine_views(
    filename=bvh_file,
    start_time=start_time,
    end_time=end_time,
    output_dir=output_dir,
    view_videos=view_videos,
    layout_config=layout_config,
    video_size=canvas_size,
    fps=output_fps
    )